<a href="https://colab.research.google.com/github/chloe-nguyenminh/Movie_Genre_Detector_Project/blob/main/Copy_of_Movie_Detector_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Movie Detector Final

In [ ]:
# Set up: Importing numpy, pandas, etc.
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from numpy.random import RandomState
from tensorflow.keras import datasets, layers, models, Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/data_no_dupl_with_existing_posters.csv')


In [ ]:
# resizing
img_width = 253
img_height = 253

image_dir = '/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/train'
image_dir2 = '/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/test'

train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=image_dir,
    image_size=(img_width, img_height),
    shuffle=True,
    batch_size=32,
    label_mode='int'
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=image_dir2,
    image_size=(img_width, img_height),
    shuffle=True,
    batch_size=32,
)

print(train_dataset)


In [ ]:
class_names = train_dataset.class_names

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Added layer of augmentation
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)), 
                                      layers.RandomRotation(0.1), layers.RandomZoom(0.1)])

In [ ]:
num_classes = len(class_names)
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

epochs=10
history = model.fit(
  train_dataset,
  batch_size=32,
  epochs=epochs,
  validation_data=test_dataset,
)